下一步是将在`Research`中构建的数据管道集成到我们的算法中。与`Research`的一个重要区别是，在回溯测试期间，随着仿真的进行，管道将每天执行，因此不需要`start_date`和`end_date`参数。

为了在算法中使用数据管道，第一步是在算法的`initialize`函数中添加对它的引用。这是通过使用`attach_pipeline`函数完成的，该函数需要两个输入参数：对Pipeline对象（使用make_pipeline构建的对象）的引用以及用于标识它的字符串名称。

如上所述，管道将在每天市场开盘之前处理数据流并生成输出。可以使用`pipeline_output`函数在`before_trading_start`中获取管道输出，`pipeline_output`函数采用我们在`initialize`中指定的管道名称，并返回由我们的管道生成的`pandas.DataFrame`。现在我们可以使用我们的`rebalance`来记录管道输出中的前10行。

现在，将我们在Research中构建的`make_pipeline`函数添加到算法中。算法不应像分析所做的那样限制资产数量，而应考虑交易总体中有尾部12月股利比的所有资产。为此，我们可以使用我们的`trailing_dividend_yield`输出的`notnull`方法创建一个过滤器，并使用`＆`运算符获得与可交易总体的交集：

In [1]:
from logbook import Logger
# Import Pipeline class and datasets
from zipline.pipeline import Pipeline
# Import built-in trading universe
from zipline.pipeline.builtin import QTradableStocksUS, trailing_dividend_yield

# Import Algorithm API functions
from zipline.api import (
    attach_pipeline,
    pipeline_output,
    schedule_function,
    date_rules,
    time_rules,
)

log = Logger('回测')


def initialize(context):
    # Attach pipeline to algorithm
    attach_pipeline(make_pipeline(), 'data_pipe')

    # Schedule rebalance function
    schedule_function(
        rebalance,
        date_rule=date_rules.week_start(),
        time_rule=time_rules.market_open())


def before_trading_start(context, data):
    # Get pipeline output and
    # store it in context
    context.output = pipeline_output('data_pipe')


def rebalance(context, data):
    # Display first 10 rows
    # of pipeline output
    log.info('\n{}'.format(context.output.head(10)))
    log.info('可选股票{}只'.format(context.output.shape[0]))


# Pipeline definition
def make_pipeline():

    base_universe = QTradableStocksUS()

    # 尾部12个月每股股利平均值/股价
    tdy = trailing_dividend_yield()

    return Pipeline(
        columns={
            'tdy': tdy,
        },
        # Set screen as the intersection between our filter
        # and trading universe
        screen=(
            base_universe
            & tdy.notnull()  # 选择非空的那一部分与可交易总体的交集
        ))

In [2]:
from zipline import run_algorithm
from zipline.research import to_dates

In [3]:
_, start_date, end_date = to_dates('2017-01-01','2018-05-18')
bundle = 'cndaily'

In [4]:
run_algorithm(
    start_date,
    end_date,
    initialize,
    100000,
    before_trading_start=before_trading_start,
    handle_data=None,
    bundle=bundle,
    bm_symbol='000300')

[2018-05-21 14:06:27.006213] INFO: 回测: 
                   tdy
平安银行(000001)  0.017967
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.012487
神州高铁(000008)  0.000000
中国宝安(000009)  0.001931
美丽生态(000010)  0.000000
深物业A(000011)  0.008009
南 玻Ａ(000012)  0.035273
沙河股份(000014)  0.000914
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:27.007185] INFO: 回测: 可选股票2081只
[2018-05-21 14:06:27.028891] INFO: 回测: 
                   tdy
平安银行(000001)  0.017908
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.011788
神州高铁(000008)  0.000000
中国宝安(000009)  0.001903
美丽生态(000010)  0.000000
深物业A(000011)  0.007407
南 玻Ａ(000012)  0.035149
沙河股份(000014)  0.000880
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:27.030050] INFO: 回测: 可选股票2075只


/home/ldf/pkg_source/czipline/src/empyrical/empyrical/stats.py:676: RuntimeWarning: invalid value encountered in true_divide
  out=out,
/home/ldf/pkg_source/czipline/src/empyrical/empyrical/stats.py:762: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


[2018-05-21 14:06:29.695079] INFO: 回测: 
                   tdy
平安银行(000001)  0.017849
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.013364
神州高铁(000008)  0.000000
中国宝安(000009)  0.001986
美丽生态(000010)  0.000000
深物业A(000011)  0.008767
南 玻Ａ(000012)  0.033417
沙河股份(000014)  0.000925
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:29.695800] INFO: 回测: 可选股票2087只
[2018-05-21 14:06:29.714459] INFO: 回测: 
                   tdy
平安银行(000001)  0.017733
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.013817
神州高铁(000008)  0.000000
中国宝安(000009)  0.002123
美丽生态(000010)  0.000000
深物业A(000011)  0.008977
南 玻Ａ(000012)  0.035778
沙河股份(000014)  0.000970
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:29.715265] INFO: 回测: 可选股票2077只
[2018-05-21 14:06:29.736624] INFO: 回测: 
                   tdy
平安银行(000001)  0.017524
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.013610
神州高铁(000008)  0.000000
中国宝安(000009)  0.002103
美丽生态(000010)  0.000000
深物业A(000011)  0.009031
南 玻Ａ(000012)  0.035180
沙河股份(000014)  0.001072
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:29.73

[2018-05-21 14:06:30.243303] INFO: 回测: 
                   tdy
平安银行(000001)  0.017412
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.013394
神州高铁(000008)  0.000000
中国宝安(000009)  0.002472
美丽生态(000010)  0.000000
深物业A(000011)  0.004000
南 玻Ａ(000012)  0.032154
沙河股份(000014)  0.001128
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:30.244761] INFO: 回测: 可选股票2144只
[2018-05-21 14:06:30.268013] INFO: 回测: 
                   tdy
平安银行(000001)  0.016156
万 科Ａ(000002)  0.020276
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.013850
神州高铁(000008)  0.000000
中国宝安(000009)  0.002356
美丽生态(000010)  0.000000
深物业A(000011)  0.004208
南 玻Ａ(000012)  0.031679
沙河股份(000014)  0.001120
[2018-05-21 14:06:30.268969] INFO: 回测: 可选股票2163只
[2018-05-21 14:06:30.291990] INFO: 回测: 
                   tdy
平安银行(000001)  0.014037
万 科Ａ(000002)  0.020333
世纪星源(000005)  0.000000
深振业Ａ(000006)  0.014201
神州高铁(000008)  0.000000
中国宝安(000009)  0.002475
美丽生态(000010)  0.000000
深物业A(000011)  0.004180
南 玻Ａ(000012)  0.030181
沙河股份(000014)  0.002209
[2018-05-21 14:06:30.29

[2018-05-21 14:06:33.112921] INFO: 回测: 
                   tdy
平安银行(000001)  0.011501
万 科Ａ(000002)  0.024121
世纪星源(000005)  0.000000
神州高铁(000008)  0.000588
中国宝安(000009)  0.009682
美丽生态(000010)  0.000000
深物业A(000011)  0.007761
南 玻Ａ(000012)  0.024213
沙河股份(000014)  0.002412
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:33.113902] INFO: 回测: 可选股票2487只
[2018-05-21 14:06:33.131852] INFO: 回测: 
                   tdy
平安银行(000001)  0.011692
万 科Ａ(000002)  0.023181
世纪星源(000005)  0.000000
神州高铁(000008)  0.000571
中国宝安(000009)  0.009682
美丽生态(000010)  0.000000
深物业A(000011)  0.007743
南 玻Ａ(000012)  0.023669
沙河股份(000014)  0.002412
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:33.132535] INFO: 回测: 可选股票2482只
[2018-05-21 14:06:33.148963] INFO: 回测: 
                   tdy
平安银行(000001)  0.011692
万 科Ａ(000002)  0.020713
世纪星源(000005)  0.000000
神州高铁(000008)  0.000585
中国宝安(000009)  0.009272
美丽生态(000010)  0.000000
深物业A(000011)  0.007242
南 玻Ａ(000012)  0.022936
沙河股份(000014)  0.002136
深康佳Ａ(000016)  0.000000
[2018-05-21 14:06:33.14

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2017-01-03 07:00:00+00:00,NaN,0.0,NaN,0.009712,NaN,NaN,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,1,[],0.0
2017-01-04 07:00:00+00:00,0.0,0.0,0.0,0.017593,0.021406,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,2,[],0.0
2017-01-05 07:00:00+00:00,0.0,0.0,0.0,0.017435,0.083084,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,3,[],0.0
2017-01-06 07:00:00+00:00,0.0,0.0,0.0,0.011356,0.115404,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,4,[],0.0
2017-01-09 07:00:00+00:00,0.0,0.0,0.0,0.016261,0.100950,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,5,[],0.0
2017-01-10 07:00:00+00:00,0.0,0.0,0.0,0.014560,0.095760,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,6,[],0.0
2017-01-11 07:00:00+00:00,0.0,0.0,0.0,0.007377,0.104382,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,7,[],0.0
2017-01-12 07:00:00+00:00,0.0,0.0,0.0,0.002279,0.102578,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,8,[],0.0
2017-01-13 07:00:00+00:00,0.0,0.0,0.0,0.002971,0.095975,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,9,[],0.0
2017-01-16 07:00:00+00:00,0.0,0.0,0.0,0.002830,0.090519,0.0,0.0,100000.0,0.0,0.0,...,0.0,0.0,0,None,100000.0,0.0,0.0,10,[],0.0


算法现在每天选择一个可交易的资产范围，并生成尾部12个月每股股利与股价的比率数据，可以使用这些数据来确定投资组合中的资产配置。在下一课中，将学习如何基于数据管道生成的每股股利与股价的比率构建最佳投资组合。